<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/YOLOv5-LIME%20(RISE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5 LIME CorneAI**

##**Setup YOLOv5**

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
%cd /content
!pip uninstall deep_utils -y
!pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git --q
!pip install torch --q
!pip install torchvision --q
!pip install -U opencv-python --q
print("[INFO] To use new installed version of opencv, the session should be restarted!!!!")

!git clone https://github.com/pooya-mohammadi/yolov5-gradcam

/content
  Preparing metadata (setup.py) ... done
[INFO] To use new installed version of opencv, the session should be restarted!!!!
Cloning into 'yolov5-gradcam'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 134 (delta 29), reused 36 (delta 13), pack-reused 68 (from 1)
Receiving objects: 100% (134/134), 6.95 MiB | 29.03 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
import os
os.chdir('/content/yolov5-gradcam')

model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/3.jpg"

#**LIME**

In [4]:
!pip install lime --q
!pip install scikit-image --q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import cv2
import time
import traceback
import torchvision
from lime import lime_image
from deep_utils.utils.box_utils.boxes import Box
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
from utils.metrics import box_iou
%matplotlib inline

# 角膜AIのクラス定義
CORNEA_CLASSES = [
    "infection",
    "normal",
    "non-infection",
    "scar",
    "tumor",
    "deposit",
    "APAC",
    "lens opacity",
    "bullous"
]

def setup_device():
    """
    GPUが利用可能な場合はGPUを、そうでない場合はCPUを設定

    Returns:
    --------
    device : torch.device
        使用するデバイス
    """
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        device = torch.device('cpu')
        print("GPU not available, using CPU")
    return device

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size=(640, 640),
                 names=CORNEA_CLASSES,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms

        # モデルのロード
        print("[INFO] Loading cornea detection model...")
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model loaded successfully")

        # モデルのクラス数を取得と確認
        self.nc = int(self.model.nc)
        print(f"[INFO] Number of classes: {self.nc}")

        # クラス名の設定と検証
        self.names = names
        if len(self.names) != self.nc:
            print(f"[WARNING] Number of class names ({len(self.names)}) does not match model classes ({self.nc})")
        print(f"[INFO] Using class names: {self.names}")

        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()

        # Cold start prevention
        print("[INFO] Performing cold start prevention...")
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)
        print("[INFO] Initialization complete")

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5
        xc = prediction[..., 4] > conf_thres

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096
        max_nms = 30000
        time_limit = 10.0
        redundant = True
        multi_label &= nc > 1
        merge = False

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, nc), device=logits.device)] * logits.shape[0]

        for xi, (x, log_) in enumerate(zip(prediction, logits)):
            x = x[xc[xi]]
            log_ = log_[xc[xi]]

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]

            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]
            if not n:
                continue
            elif n > max_nms:
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]

            c = x[:, 5:6] * (0 if agnostic else max_wh)
            boxes, scores = x[:, :4] + c, x[:, 4]
            i = torchvision.ops.nms(boxes, scores, iou_thres)

            if i.shape[0] > max_det:
                i = i[:max_det]

            output[xi] = x[i]
            logits_output[xi] = log_[i]

            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        try:
            prediction, logits, _ = self.model(img, augment=False)
            prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                          classes=None,
                                                          agnostic=self.agnostic)

            batch_size = img.shape[0]
            self.boxes = [[] for _ in range(batch_size)]
            self.class_names = [[] for _ in range(batch_size)]
            self.classes = [[] for _ in range(batch_size)]
            self.confidences = [[] for _ in range(batch_size)]

            for i, det in enumerate(prediction):
                if len(det):
                    for *xyxy, conf, cls in det:
                        xyxy[0] = max(0, xyxy[0])
                        xyxy[1] = max(0, xyxy[1])
                        xyxy[2] = min(self.img_size[1], xyxy[2])
                        xyxy[3] = min(self.img_size[0], xyxy[3])

                        bbox = Box.box2box(xyxy,
                                           in_source=Box.BoxSource.Torch,
                                           to_source=Box.BoxSource.Numpy,
                                           return_int=True)

                        self.boxes[i].append(bbox)
                        self.confidences[i].append(float(conf.item()))
                        cls_idx = int(cls.item())

                        if cls_idx >= len(self.names):
                            print(f"[WARNING] Class index {cls_idx} is out of range")
                            cls_idx = 0

                        self.classes[i].append(cls_idx)
                        self.class_names[i].append(self.names[cls_idx])

            return [self.boxes, self.classes, self.class_names, self.confidences], logits

        except Exception as e:
            print(f"Error in forward pass: {e}")
            traceback.print_exc()
            return [[[]], [[]], [[]], [[]]], None

    def preprocessing(self, img):
        try:
            if len(img.shape) != 4:
                img = np.expand_dims(img, axis=0)
            im0 = img.astype(np.uint8)
            img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
            img = img.transpose((0, 3, 1, 2))
            img = np.ascontiguousarray(img)
            img = torch.from_numpy(img).to(self.device)
            img = img / 255.0
            return img
        except Exception as e:
            print(f"Error in preprocessing: {e}")
            traceback.print_exc()
            return None

class YOLOLimeExplainer:
    def __init__(self, yolo_model, device='cuda', img_size=(640, 640)):
        self.model = yolo_model
        self.device = device
        self.img_size = img_size
        self.explainer = lime_image.LimeImageExplainer()

    def predict_fn(self, images):
        try:
            batch_predictions = []
            for img in images:
                processed_img = self.model.preprocessing(np.expand_dims(img, 0))
                if processed_img is None:
                    raise ValueError("Failed to preprocess image")

                with torch.no_grad():
                    predictions, _ = self.model(processed_img)

                class_scores = np.zeros(len(CORNEA_CLASSES))

                if predictions[0][0]:
                    for cls_idx, conf in zip(predictions[1][0], predictions[3][0]):
                        if cls_idx < len(CORNEA_CLASSES):
                            class_scores[cls_idx] = max(class_scores[cls_idx], conf)

                batch_predictions.append(class_scores)

            return np.array(batch_predictions)

        except Exception as e:
            print(f"Error in prediction_fn: {e}")
            traceback.print_exc()
            return np.zeros((len(images), len(CORNEA_CLASSES)))

    def explain_instance(self, image, num_samples=1000, top_labels=5):
        try:
            if isinstance(image, Image.Image):
                image = np.array(image)
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif image.shape[2] == 4:
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

            explanation = self.explainer.explain_instance(
                image,
                self.predict_fn,
                labels=range(len(CORNEA_CLASSES)),
                top_labels=top_labels,
                hide_color=0,
                num_samples=num_samples
            )
            return explanation

        except Exception as e:
            print(f"Error in explain_instance: {e}")
            traceback.print_exc()
            return None

def visualize_results(explanation, image, class_names=CORNEA_CLASSES, save_path=None):
    """
    Visualize LIME explanation results overlaid on the original image

    Args:
        explanation: LIME explanation object
        image: Original image array
        class_names: List of class names
        save_path: Optional path to save the visualization
    """
    try:
        # Get predictions
        prediction = explanation.predict_fn(np.array([image]))[0]

        # Sort labels by prediction confidence
        sorted_labels = sorted(range(len(prediction)),
                             key=lambda x: prediction[x],
                             reverse=True)[:2]

        # Create a figure with subplots
        plt.figure(figsize=(15, 7))

        for idx, label in enumerate(sorted_labels):
            plt.subplot(1, len(sorted_labels), idx + 1)

            # Get the original image and mask from LIME
            mask = explanation.local_exp[label]

            # Convert the sparse mask to a dense array
            dense_mask = np.zeros(explanation.segments.shape, dtype=float)
            for i, v in mask:
                dense_mask[explanation.segments == i] = v

            # Normalize the mask to [0, 1] range
            if dense_mask.max() != dense_mask.min():
                dense_mask = (dense_mask - dense_mask.min()) / (dense_mask.max() - dense_mask.min())

            # Create a colormap (red for positive contributions)
            heatmap = np.zeros((dense_mask.shape[0], dense_mask.shape[1], 4))
            heatmap[:, :, 2] = dense_mask  # Blue channel
            heatmap[:, :, 3] = dense_mask * 1.0  # Alpha channel

            # Display original image
            plt.imshow(image, alpha=0.8)

            # Overlay heatmap
            plt.imshow(heatmap, alpha=0.6)

            class_name = class_names[label] if label < len(class_names) else f"Unknown Class {label}"
            plt.title(f'{class_name}\nConfidence: {prediction[label]:.3f}', fontsize=12)
            plt.axis('off')

        plt.tight_layout()

        # if save_path:
        #     plt.savefig(save_path, bbox_inches='tight', dpi=300)
        #     print(f"Visualization saved to {save_path}")

        plt.show()

        print("\nDetailed Confidence Scores:")
        for label in sorted_labels:
            print(f"{class_names[label]}: {prediction[label]:.3f}")

    except Exception as e:
        print(f"Error in visualization: {e}")
        traceback.print_exc()

class YOLOLimeExplainer:
    def __init__(self, yolo_model, device='cuda', img_size=(640, 640)):
        self.model = yolo_model
        self.device = device
        self.img_size = img_size
        self.explainer = lime_image.LimeImageExplainer()

    def predict_fn(self, images):
        """
        Modified prediction function with improved image processing
        """
        try:
            batch_predictions = []
            for img in images:
                # Normalize image if needed
                if img.max() > 1.0:
                    img = img / 255.0

                # Convert image format if needed
                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                elif img.shape[2] == 4:
                    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

                # Ensure image is uint8 for preprocessing
                img_uint8 = (img * 255).astype(np.uint8)
                processed_img = self.model.preprocessing(np.expand_dims(img_uint8, 0))

                if processed_img is None:
                    raise ValueError("Failed to preprocess image")

                with torch.no_grad():
                    predictions, logits = self.model(processed_img)

                class_scores = np.zeros(len(CORNEA_CLASSES))

                if predictions[0][0]:  # If there are any detections
                    if logits is not None and len(logits[0]) > 0:
                        # Use logits if available
                        probs = torch.nn.functional.softmax(logits[0], dim=1)
                        class_scores = probs.mean(dim=0).cpu().numpy()
                    else:
                        # Fallback to confidence scores
                        for cls_idx, conf in zip(predictions[1][0], predictions[3][0]):
                            if cls_idx < len(CORNEA_CLASSES):
                                class_scores[cls_idx] = max(class_scores[cls_idx], conf)

                batch_predictions.append(class_scores)

            return np.array(batch_predictions)

        except Exception as e:
            print(f"Error in prediction_fn: {e}")
            traceback.print_exc()
            return np.zeros((len(images), len(CORNEA_CLASSES)))

    def explain_instance(self, image, num_samples=1000, top_labels=5):
        """
        Generate LIME explanation with improved image handling
        """
        try:
            # Convert PIL Image to numpy array if needed
            if isinstance(image, Image.Image):
                image = np.array(image)

            # Normalize image if needed
            if image.max() > 1.0:
                image = image.astype(float) / 255.0

            # Convert image format if needed
            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            elif image.shape[2] == 4:
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

            # Generate explanation
            explanation = self.explainer.explain_instance(
                image,
                self.predict_fn,
                labels=range(len(CORNEA_CLASSES)),
                top_labels=top_labels,
                hide_color=0,
                num_samples=num_samples
            )

            # Store predict_fn for later use
            explanation.predict_fn = self.predict_fn

            return explanation

        except Exception as e:
            print(f"Error in explain_instance: {e}")
            traceback.print_exc()
            return None

def run_lime_analysis(model_path, img_path, num_samples=500, save_path=None):
    """
    Run LIME analysis with improved visualization

    Args:
        model_path: Path to the YOLO model weights
        img_path: Path to the input image
        num_samples: Number of samples for LIME analysis
        save_path: Optional path to save the visualization
    """
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")

        print("Loading YOLO model...")
        yolo_model = YOLOV5TorchObjectDetector(
            model_weight=model_path,
            device=device,
            img_size=(640, 640),
            names=CORNEA_CLASSES
        )

        print("Initializing LIME explainer...")
        lime_explainer = YOLOLimeExplainer(yolo_model)

        print("Loading and processing image...")
        image = Image.open(img_path)
        image_array = np.array(image)

        # Normalize image if needed
        if image_array.max() > 1.0:
            image_array = image_array.astype(float) / 255.0

        print(f"Running LIME analysis with {num_samples} samples...")
        explanation = lime_explainer.explain_instance(
            image_array,
            num_samples=num_samples,
            top_labels=3
        )

        if explanation is None:
            print("Failed to generate explanation")
            return None

        print("Visualizing results...")
        visualize_results(explanation, image_array, save_path=save_path)

        return explanation

    except Exception as e:
        print(f"Error in run_lime_analysis: {e}")
        traceback.print_exc()
        return None

# Usage example:
if __name__ == "__main__":
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/1.jpg"
    save_path = "lime_explanation.png"  # Optional

    explanation = run_lime_analysis(
        model_path=model_path,
        img_path=img_path,
        num_samples=500,
        save_path=save_path
    )

#**RISE**

In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import cv2
import time
import traceback
import torchvision
from deep_utils.utils.box_utils.boxes import Box
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
from utils.metrics import box_iou
%matplotlib inline

# 角膜AIのクラス定義
CORNEA_CLASSES = [
    "infection",
    "normal",
    "non-infection",
    "scar",
    "tumor",
    "deposit",
    "APAC",
    "lens opacity",
    "bullous"
]

def setup_device():
    """
    GPUが利用可能な場合はGPUを、そうでない場合はCPUを設定

    Returns:
    --------
    device : torch.device
        使用するデバイス
    """
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        device = torch.device('cpu')
        print("GPU not available, using CPU")
    return device

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size=(640, 640),
                 names=CORNEA_CLASSES,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms

        # モデルのロード
        print("[INFO] Loading cornea detection model...")
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model loaded successfully")

        # モデルのクラス数を取得と確認
        self.nc = int(self.model.nc)
        print(f"[INFO] Number of classes: {self.nc}")

        # クラス名の設定と検証
        self.names = names
        if len(self.names) != self.nc:
            print(f"[WARNING] Number of class names ({len(self.names)}) does not match model classes ({self.nc})")
        print(f"[INFO] Using class names: {self.names}")

        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()

        # Cold start prevention
        print("[INFO] Performing cold start prevention...")
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)
        print("[INFO] Initialization complete")

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5
        xc = prediction[..., 4] > conf_thres

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096
        max_nms = 30000
        time_limit = 10.0
        redundant = True
        multi_label &= nc > 1
        merge = False

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, nc), device=logits.device)] * logits.shape[0]

        for xi, (x, log_) in enumerate(zip(prediction, logits)):
            x = x[xc[xi]]
            log_ = log_[xc[xi]]

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]

            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]
            if not n:
                continue
            elif n > max_nms:
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]

            c = x[:, 5:6] * (0 if agnostic else max_wh)
            boxes, scores = x[:, :4] + c, x[:, 4]
            i = torchvision.ops.nms(boxes, scores, iou_thres)

            if i.shape[0] > max_det:
                i = i[:max_det]

            output[xi] = x[i]
            logits_output[xi] = log_[i]

            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        try:
            prediction, logits, _ = self.model(img, augment=False)
            prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                          classes=None,
                                                          agnostic=self.agnostic)

            batch_size = img.shape[0]
            self.boxes = [[] for _ in range(batch_size)]
            self.class_names = [[] for _ in range(batch_size)]
            self.classes = [[] for _ in range(batch_size)]
            self.confidences = [[] for _ in range(batch_size)]

            for i, det in enumerate(prediction):
                if len(det):
                    for *xyxy, conf, cls in det:
                        xyxy[0] = max(0, xyxy[0])
                        xyxy[1] = max(0, xyxy[1])
                        xyxy[2] = min(self.img_size[1], xyxy[2])
                        xyxy[3] = min(self.img_size[0], xyxy[3])

                        bbox = Box.box2box(xyxy,
                                           in_source=Box.BoxSource.Torch,
                                           to_source=Box.BoxSource.Numpy,
                                           return_int=True)

                        self.boxes[i].append(bbox)
                        self.confidences[i].append(float(conf.item()))
                        cls_idx = int(cls.item())

                        if cls_idx >= len(self.names):
                            print(f"[WARNING] Class index {cls_idx} is out of range")
                            cls_idx = 0

                        self.classes[i].append(cls_idx)
                        self.class_names[i].append(self.names[cls_idx])

            return [self.boxes, self.classes, self.class_names, self.confidences], logits

        except Exception as e:
            print(f"Error in forward pass: {e}")
            traceback.print_exc()
            return [[[]], [[]], [[]], [[]]], None

    def preprocessing(self, img):
        try:
            if len(img.shape) != 4:
                img = np.expand_dims(img, axis=0)
            im0 = img.astype(np.uint8)
            img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
            img = img.transpose((0, 3, 1, 2))
            img = np.ascontiguousarray(img)
            img = torch.from_numpy(img).to(self.device)
            img = img / 255.0
            return img
        except Exception as e:
            print(f"Error in preprocessing: {e}")
            traceback.print_exc()
            return None

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import time
import traceback

import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import cv2


class YOLORISE:
    def __init__(self, model, input_size=(640, 640), gpu_batch=32):
        self.model = model
        self.input_size = input_size
        self.gpu_batch = gpu_batch
        self.N = None
        self.masks = None
        self.device = next(model.parameters()).device

    def generate_masks(self, N=6000, s=8, p1=0.1, savepath=None, target_shape=None):
        """Generate random binary masks for RISE"""
        if target_shape is None:
            target_shape = self.input_size

        cell_height = int(np.ceil(target_shape[0] / s))
        cell_width = int(np.ceil(target_shape[1] / s))

        self.N = N
        self.masks = np.zeros((N, target_shape[0], target_shape[1]), dtype=np.float32)

        print(f"Generating {N} masks for shape {target_shape} with cell size {cell_height}x{cell_width}...")
        for i in tqdm(range(N), desc="Generating masks"):
            small_mask = np.random.choice(
                [0, 1],
                size=(cell_height, cell_width),
                p=[1-p1, p1]
            ).astype(np.float32)

            small_mask_3ch = np.repeat(small_mask[:, :, np.newaxis], 3, axis=2)

            mask = cv2.resize(
                small_mask_3ch,
                (target_shape[1], target_shape[0]),
                interpolation=cv2.INTER_LINEAR
            )

            self.masks[i] = (mask[:, :, 0] > 0.5).astype(np.float32)

        if savepath:
            np.save(savepath, self.masks)
            print(f"Masks saved to {savepath} with shape {self.masks.shape}")

    def load_masks(self, path, target_shape=None):
        """Load pre-generated masks with validation"""
        try:
            loaded_masks = np.load(path)

            if target_shape is None:
                target_shape = self.input_size

            expected_shape = (self.N if self.N is not None else 6000, target_shape[0], target_shape[1])

            if loaded_masks.shape != expected_shape:
                print(f"Warning: Loaded masks shape {loaded_masks.shape} differs from expected {expected_shape}")
                print("Regenerating masks...")
                self.generate_masks(N=expected_shape[0], savepath=path, target_shape=target_shape)
                return

            self.masks = loaded_masks
            self.N = loaded_masks.shape[0]
            print(f"Loaded {self.N} masks with shape {self.masks.shape}")

        except (ValueError, IOError) as e:
            print(f"Error loading masks: {e}")
            print("Regenerating masks...")
            self.generate_masks(N=6000 if self.N is None else self.N, savepath=path, target_shape=target_shape)

    def __call__(self, images):
        """
        Generate saliency maps for YOLOv5 predictions

        Args:
            images: Input images tensor [B, C, H, W]
        """
        if self.masks is None:
            raise ValueError("Masks not generated. Call generate_masks() first.")

        batch_size = images.shape[0]
        input_shape = (images.shape[2], images.shape[3])
        print(f"Processing batch of {batch_size} images with shape {images.shape}")

        # Regenerate masks if input shape doesn't match
        if self.masks.shape[1:] != input_shape:
            print(f"Regenerating masks for input shape {input_shape}")
            mask_path = os.path.join('.', f'rise_masks_{input_shape[0]}x{input_shape[1]}.npy')
            self.generate_masks(N=self.N if self.N is not None else 6000, target_shape=input_shape, savepath=mask_path)

        # Get baseline predictions
        with torch.no_grad():
            baseline_preds, baseline_logits = self.model(images)

        # Initialize saliency maps
        class_saliency = torch.zeros(
            (batch_size, len(self.model.names), *input_shape),
            device=self.device
        )
        box_saliency = torch.zeros(
            (batch_size, *input_shape),
            device=self.device
        )

        # Convert masks to tensor
        masks_tensor = torch.from_numpy(self.masks).float().to(self.device)

        # Process masks in batches
        for i in range(0, self.N, self.gpu_batch):
            batch_end = min(i + self.gpu_batch, self.N)
            batch_masks = masks_tensor[i:batch_end]

            # Apply masks to images
            masked_batch = []
            for img in images:
                masked_images = img.unsqueeze(0) * batch_masks.view(-1, 1, *input_shape)
                masked_batch.append(masked_images)

            masked_batch = torch.cat(masked_batch, dim=0)

            # Get predictions for masked images
            with torch.no_grad():
                predictions, logits = self.model(masked_batch)

            # Process each image in the batch
            for b in range(batch_size):
                # Get predictions for current image
                base_boxes = baseline_preds[0][b]
                base_classes = baseline_preds[1][b]
                base_scores = baseline_preds[3][b]

                if len(base_classes) > 0:
                    # Update class saliency
                    for cls, score in zip(base_classes, base_scores):
                        cls_idx = int(cls)
                        if cls_idx < len(self.model.names):
                            # マスクの形状を明示的に合わせる
                            for mask_idx in range(len(batch_masks)):
                                class_saliency[b, cls_idx] += batch_masks[mask_idx] * float(score)

                    # Update box saliency
                    for box in base_boxes:
                        score = self._calculate_box_score(box)
                        # マスクの形状を明示的に合わせる
                        for mask_idx in range(len(batch_masks)):
                            box_saliency[b] += batch_masks[mask_idx] * score

        # Normalize saliency maps
        class_saliency = self._normalize_saliency(class_saliency)
        box_saliency = self._normalize_saliency(box_saliency)

        return {
            'class_saliency': class_saliency,
            'box_saliency': box_saliency
        }

    def _calculate_box_score(self, box):
        """Calculate score for bounding box based on area"""
        x1, y1, x2, y2 = box
        area = (x2 - x1) * (y2 - y1)
        return float(area) / (self.input_size[0] * self.input_size[1])

    def _normalize_saliency(self, saliency):
        """Normalize saliency maps to [0, 1] range"""
        saliency_min = torch.min(saliency)
        saliency_max = torch.max(saliency)

        if saliency_max > saliency_min:
            saliency = (saliency - saliency_min) / (saliency_max - saliency_min)

        return saliency

def apply_heatmap(image, saliency_map, alpha=0.4):
    """
    Apply saliency map as a heatmap overlay on the original image

    Args:
        image: Original image (H, W, 3) in RGB format
        saliency_map: Tensor containing saliency values (H, W)
        alpha: Transparency of the heatmap overlay (0-1)

    Returns:
        overlayed_image: Image with heatmap overlay
    """
    # Convert saliency map to numpy if it's a tensor
    if torch.is_tensor(saliency_map):
        saliency_map = saliency_map.cpu().numpy()

    # Normalize saliency map to 0-1
    saliency_norm = (saliency_map - np.min(saliency_map)) / (np.max(saliency_map) - np.min(saliency_map))

    # Convert to uint8 for cv2
    heatmap = (saliency_norm * 255).astype(np.uint8)

    # Apply colormap
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Convert to RGB
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Resize heatmap if sizes don't match
    if image.shape[:2] != heatmap.shape[:2]:
        heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

    # Overlay heatmap on original image
    overlayed = cv2.addWeighted(image, 1-alpha, heatmap, alpha, 0)

    return overlayed

def visualize_results(image, results, save_path=None):
    """
    Visualize RISE analysis results with heatmaps

    Args:
        image: Original image array
        results: Dictionary containing RISE analysis results
        save_path: Optional path to save the visualization
    """
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('RISE Analysis Results', fontsize=16)

    # Plot original image with detections
    axes[0, 0].imshow(image)
    axes[0, 0].set_title('Original Image with Detections')

    # Draw bounding boxes
    for box, cls_name, conf in zip(results['boxes'], results['class_names'], results['confidences']):
        x1, y1, x2, y2 = box
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2)
        axes[0, 0].add_patch(rect)
        axes[0, 0].text(
            x1, y1-5,
            f'{cls_name}: {conf:.2f}',
            color='red',
            fontsize=8,
            bbox=dict(facecolor='white', alpha=0.7)
        )

    # Plot box saliency heatmap
    box_saliency = results['saliency_maps']['box_saliency'][0].cpu()
    box_heatmap = apply_heatmap(image, box_saliency)
    axes[0, 1].imshow(box_heatmap)
    axes[0, 1].set_title('Box Saliency Heatmap')

    # Plot combined class saliency heatmap
    class_saliency = results['saliency_maps']['class_saliency'][0]
    combined_saliency = class_saliency.mean(dim=0).cpu()
    combined_heatmap = apply_heatmap(image, combined_saliency)
    axes[0, 2].imshow(combined_heatmap)
    axes[0, 2].set_title('Combined Class Saliency Heatmap')

    # Plot top 3 class-specific saliency heatmaps
    top_classes = torch.topk(torch.tensor(results['confidences']), k=min(3, len(results['confidences'])))

    for idx, (cls_idx, conf) in enumerate(zip(top_classes.indices, top_classes.values)):
        class_heatmap = apply_heatmap(image, class_saliency[cls_idx].cpu())
        axes[1, idx].imshow(class_heatmap)
        axes[1, idx].set_title(f'Class Saliency: {results["class_names"][idx]}\nConf: {conf:.2f}')

    # Turn off axes
    for ax in axes.flat:
        ax.axis('off')

    # Adjust layout and save
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)

    plt.close()

def display_inference_results(image, predictions, class_names, save_path=None):
    """
    Display inference results on the image

    Args:
        image: Original image array
        predictions: List containing [boxes, classes, class_names, confidences]
        class_names: List of class names
        save_path: Optional path to save the visualization
    """
    # Create figure
    plt.figure(figsize=(12, 8))
    plt.imshow(image)

    # Draw boxes and labels
    boxes, classes, pred_class_names, confidences = predictions

    for box, cls_name, conf in zip(boxes[0], pred_class_names[0], confidences[0]):
        x1, y1, x2, y2 = box
        plt.gca().add_patch(plt.Rectangle(
            (x1, y1), x2-x1, y2-y1,
            fill=False, color='red', linewidth=2
        ))
        plt.text(
            x1, y1-5,
            f'{cls_name}: {conf:.2f}',
            color='red',
            fontsize=10,
            bbox=dict(facecolor='white', alpha=0.7)
        )

    plt.title('YOLOv5 Detection Results')
    plt.axis('off')

    if save_path:
        inference_path = os.path.splitext(save_path)[0] + '_inference.png'
        plt.savefig(inference_path, bbox_inches='tight', dpi=300)

    plt.show()
    plt.close()

def run_rise_analysis(model_path, img_path, num_masks=6000, mask_size=8, mask_prob=0.1, save_path=None):
    """Run RISE analysis on a YOLOv5 model and image"""
    try:
        start_time = time.time()
        device = setup_device()

        # Load YOLO model
        print("Loading YOLOv5 model...")
        yolo_model = YOLOV5TorchObjectDetector(
            model_weight=model_path,
            device=device,
            img_size=(640, 640),
            names=CORNEA_CLASSES
        )

        # Initialize RISE explainer
        print("Initializing RISE explainer...")
        rise_explainer = YOLORISE(yolo_model, input_size=(640, 640))

        # Generate or load masks
        mask_path = os.path.join(os.path.dirname(save_path) if save_path else '.', 'rise_masks.npy')

        if os.path.exists(mask_path):
            print("Loading pre-generated masks...")
            rise_explainer.load_masks(mask_path)
        else:
            print(f"Generating {num_masks} masks...")
            rise_explainer.generate_masks(N=num_masks, s=mask_size, p1=mask_prob, savepath=mask_path)

        # Load and preprocess image
        print("Loading and preprocessing image...")
        original_image = Image.open(img_path)
        image_array = np.array(original_image)

        if len(image_array.shape) == 2:
            image_array = cv2.cvtColor(image_array, cv2.COLOR_GRAY2RGB)
        elif image_array.shape[2] == 4:
            image_array = cv2.cvtColor(image_array, cv2.COLOR_RGBA2RGB)

        processed_image = yolo_model.preprocessing(np.expand_dims(image_array, 0))

        if processed_image is None:
            raise ValueError("Failed to preprocess image")

        print(f"Input image shape: {processed_image.shape}")

        # Get baseline predictions
        print("Running inference...")
        with torch.no_grad():
            predictions, logits = yolo_model(processed_image)

        # Display inference results
        print("Displaying inference results...")
        display_inference_results(image_array, predictions, CORNEA_CLASSES, save_path)

        # Run RISE analysis
        print("Running RISE analysis...")
        saliency_maps = rise_explainer(processed_image)

        results = {
            'boxes': predictions[0][0],
            'classes': predictions[1][0],
            'class_names': predictions[2][0],
            'confidences': predictions[3][0],
            'saliency_maps': saliency_maps,
            'processing_time': time.time() - start_time
        }

        if save_path:
            print("Generating visualizations...")
            # Generate original visualization
            _generate_detailed_visualization(
                image_array,
                results,
                saliency_maps,
                class_names=CORNEA_CLASSES,
                save_path=save_path
            )

            # Generate heatmap visualization
            heatmap_path = os.path.splitext(save_path)[0] + '_heatmap.png'
            visualize_results(image_array, results, save_path=heatmap_path)

        return results

    except Exception as e:
        print(f"Error in RISE analysis: {e}")
        traceback.print_exc()
        return None

def _generate_detailed_visualization(image, results, saliency_maps, class_names, save_path):
    """
    Generate detailed visualization of RISE analysis results with proper cleanup
    """
    try:
        fig = plt.figure(figsize=(20, 15))

        # Plot original image with detections
        plt.subplot(231)
        plt.title('Original Image with Detections')
        plt.imshow(image)

        # Draw bounding boxes
        for box, cls_name, conf in zip(results['boxes'], results['class_names'], results['confidences']):
            x1, y1, x2, y2 = box
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                fill=False, color='red', linewidth=2
            ))
            plt.text(
                x1, y1-5,
                f'{cls_name}: {conf:.2f}',
                color='red',
                fontsize=8,
                bbox=dict(facecolor='white', alpha=0.7)
            )
        plt.axis('off')

        # Plot class saliency for top classes
        class_saliency = saliency_maps['class_saliency']
        top_classes = torch.topk(torch.tensor(results['confidences']), k=min(3, len(results['confidences'])))

        for idx, (cls_idx, conf) in enumerate(zip(top_classes.indices, top_classes.values)):
            plt.subplot(2, 3, idx+2)
            plt.title(f'Class Saliency: {class_names[cls_idx]}\nConf: {conf:.2f}')
            plt.imshow(image, alpha=0.6)
            plt.imshow(
                class_saliency[0, cls_idx].cpu(),
                cmap='jet',
                alpha=0.4
            )
            plt.axis('off')

        # Plot box saliency
        plt.subplot(234)
        plt.title('Box Saliency')
        plt.imshow(image, alpha=0.6)
        plt.imshow(
            saliency_maps['box_saliency'][0].cpu(),
            cmap='jet',
            alpha=0.4
        )
        plt.axis('off')

        # Plot combined saliency
        plt.subplot(235)
        plt.title('Combined Saliency')
        combined_saliency = (
            saliency_maps['box_saliency'][0] +
            class_saliency[0].mean(dim=0)
        ) / 2
        plt.imshow(image, alpha=0.6)
        plt.imshow(
            combined_saliency.cpu(),
            cmap='jet',
            alpha=0.4
        )
        plt.axis('off')

        # Add processing information
        plt.subplot(236)
        plt.axis('off')
        info_text = f"""Processing Information:

    Total Time: {results['processing_time']:.2f}s
    Number of Detections: {len(results['boxes'])}

    Top Detections:
    """
        for cls_name, conf in zip(results['class_names'], results['confidences']):
            info_text += f"- {cls_name}: {conf:.2f}\n"

        plt.text(0.1, 0.5, info_text, fontsize=10, va='center')

        # Save and cleanup
        plt.tight_layout()
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        plt.close(fig)
        plt.close('all')

    except Exception as e:
        print(f"Error in visualization: {e}")
        plt.close('all')
        raise



def batch_process_images(model_path, image_dir, output_dir, batch_size=1, **rise_params):
    """
    Process multiple images with RISE analysis

    Args:
        model_path: Path to YOLOv5 model weights
        image_dir: Directory containing images
        output_dir: Directory to save results
        batch_size: Number of images to process in parallel
        rise_params: Additional parameters for RISE analysis
    """
    try:
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Get list of images
        image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Process images
        for i in tqdm(range(0, len(image_files), batch_size), desc="Processing images"):
            batch_files = image_files[i:i+batch_size]

            for img_file in batch_files:
                img_path = os.path.join(image_dir, img_file)
                save_path = os.path.join(output_dir, f'rise_{os.path.splitext(img_file)[0]}.png')

                # Run RISE analysis
                results = run_rise_analysis(
                    model_path=model_path,
                    img_path=img_path,
                    save_path=save_path,
                    **rise_params
                )

                if results is None:
                    print(f"Failed to process {img_file}")
                    continue

                # Save raw results if needed
                results_path = os.path.join(output_dir, f'rise_{os.path.splitext(img_file)[0]}_results.npz')
                np.savez_compressed(
                    results_path,
                    boxes=results['boxes'],
                    classes=results['classes'],
                    confidences=results['confidences'],
                    class_saliency=results['saliency_maps']['class_saliency'].cpu().numpy(),
                    box_saliency=results['saliency_maps']['box_saliency'].cpu().numpy()
                )

    except Exception as e:
        print(f"Error in batch processing: {e}")
        traceback.print_exc()

# Example usage
if __name__ == "__main__":
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/1.jpg"
    save_path = "rise_analysis.png"

    results = run_rise_analysis(
        model_path=model_path,
        img_path=img_path,
        save_path=save_path,
        num_masks=6000,
        mask_size=8,
        mask_prob=0.1
    )


    # # Batch processing
    # batch_process_images(
    #     model_path=model_path,
    #     image_dir="path/to/image/directory",
    #     output_dir="path/to/output/directory",
    #     batch_size=1,
    #     num_masks=6000,
    #     mask_size=8,
    #     mask_prob=0.1
    # )



In [19]:
import os
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import time
import traceback
import torchvision
from tqdm import tqdm

# YOLOv5のクラス定義
CORNEA_CLASSES = [
    "infection",
    "normal",
    "non-infection",
    "scar",
    "tumor",
    "deposit",
    "APAC",
    "lens opacity",
    "bullous"
]

def setup_device():
    """GPUが利用可能な場合はGPUを、そうでない場合はCPUを設定"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    else:
        device = torch.device('cpu')
        print("GPU not available, using CPU")
    return device

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size=(640, 640),
                 names=CORNEA_CLASSES,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45):
        """YOLOv5モデルの初期化"""
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.names = names

        # モデルのロード
        print("[INFO] Loading model...")
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model loaded successfully")

        # モデルのクラス数を取得と確認
        self.nc = int(self.model.nc)
        print(f"[INFO] Number of classes: {self.nc}")

        if len(self.names) != self.nc:
            print(f"[WARNING] Number of class names ({len(self.names)}) does not match model classes ({self.nc})")
        print(f"[INFO] Using class names: {self.names}")

        self.model.to(device)
        if self.mode == 'eval':
            self.model.eval()

        # Cold start prevention
        print("[INFO] Performing cold start prevention...")
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)
        print("[INFO] Initialization complete")

    def preprocessing(self, img):
        """画像の前処理"""
        try:
            if len(img.shape) != 4:
                img = np.expand_dims(img, axis=0)
            im0 = img.astype(np.uint8)
            img = np.array([letterbox(im, new_shape=self.img_size)[0] for im in im0])
            img = img.transpose((0, 3, 1, 2))
            img = np.ascontiguousarray(img)
            img = torch.from_numpy(img).to(self.device)
            img = img / 255.0
            return img
        except Exception as e:
            print(f"Error in preprocessing: {e}")
            traceback.print_exc()
            return None

    def forward(self, img):
        """モデルの推論実行"""
        try:
            prediction = self.model(img)[0]

            # NMSの適用
            prediction = non_max_suppression(
                prediction,
                conf_thres=self.confidence,
                iou_thres=self.iou_thresh
            )

            batch_size = img.shape[0]
            self.boxes = [[] for _ in range(batch_size)]
            self.class_names = [[] for _ in range(batch_size)]
            self.classes = [[] for _ in range(batch_size)]
            self.confidences = [[] for _ in range(batch_size)]

            for i, det in enumerate(prediction):
                if len(det):
                    # 信頼度でソート
                    det = det[det[:, 4].argsort(descending=True)]

                    for *xyxy, conf, cls in det:
                        # バウンディングボックスの座標を画像サイズ内に制限
                        xyxy[0] = max(0, xyxy[0])
                        xyxy[1] = max(0, xyxy[1])
                        xyxy[2] = min(self.img_size[1], xyxy[2])
                        xyxy[3] = min(self.img_size[0], xyxy[3])

                        self.boxes[i].append(xyxy)
                        self.confidences[i].append(float(conf.item()))
                        cls_idx = int(cls.item())

                        if cls_idx >= len(self.names):
                            print(f"[WARNING] Class index {cls_idx} is out of range")
                            cls_idx = 0

                        self.classes[i].append(cls_idx)
                        self.class_names[i].append(self.names[cls_idx])

            return [self.boxes, self.classes, self.class_names, self.confidences]

        except Exception as e:
            print(f"Error in forward pass: {e}")
            traceback.print_exc()
            return [[[]], [[]], [[]], [[]]]

class YOLORISE:
    def __init__(self, model, N=6000, s=8, p1=0.1, batch_size=32):
        """YOLORISEの初期化"""
        self.model = model
        self.N = N
        self.s = s
        self.p1 = p1
        self.batch_size = batch_size
        self.device = next(model.parameters()).device
        self.masks = None
        # input_sizeはイニシャライザから削除し、実際の入力画像のサイズに基づいて決定する

    def generate_masks(self, savepath=None, image_size=(640, 640)):
        """RISEマスクの生成"""
        print(f"Generating {self.N} masks for size {image_size}...")

        # マスクのグリッドサイズを計算
        cell_height = int(np.ceil(image_size[0] / self.s))
        cell_width = int(np.ceil(image_size[1] / self.s))

        # マスクの生成
        self.masks = np.zeros((self.N, image_size[0], image_size[1]), dtype=np.float32)

        for i in tqdm(range(self.N), desc="Generating masks"):
            # 小さいサイズのマスクを生成
            small_mask = np.random.choice(
                [0, 1],
                size=(cell_height, cell_width),
                p=[1-self.p1, self.p1]
            ).astype(np.float32)

            # マスクをリサイズ
            mask = cv2.resize(
                small_mask,
                (self.input_size[1], self.input_size[0]),
                interpolation=cv2.INTER_LINEAR
            )

            self.masks[i] = (mask > 0.5).astype(np.float32)

        if savepath:
            np.save(savepath, self.masks)
            print(f"Masks saved to {savepath}")

    def load_masks(self, path):
        """保存されたマスクの読み込み"""
        try:
            self.masks = np.load(path)
            self.N = self.masks.shape[0]
            print(f"Loaded {self.N} masks with shape {self.masks.shape}")
        except Exception as e:
            print(f"Error loading masks: {e}")
            print("Regenerating masks...")
            self.generate_masks(savepath=path)

    def __call__(self, image):
        """RISEの実行"""
        if self.masks is None:
            raise ValueError("Masks not generated. Call generate_masks() first.")

        # 入力画像のサイズを取得
        _, _, H, W = image.shape

        # マスクをリサイズ
        if self.masks.shape[1:] != (H, W):
            print(f"Resizing masks from {self.masks.shape[1:]} to {(H, W)}")
            resized_masks = np.zeros((self.N, H, W), dtype=np.float32)
            for i in range(self.N):
                resized_masks[i] = cv2.resize(self.masks[i], (W, H), interpolation=cv2.INTER_LINEAR)
            masks_tensor = torch.from_numpy(resized_masks).float().to(self.device)
        else:
            masks_tensor = torch.from_numpy(self.masks).float().to(self.device)

        saliency_map = torch.zeros((H, W), device=self.device)

        # バッチ処理でマスクを適用
        for i in range(0, self.N, self.batch_size):
            batch_masks = masks_tensor[i:i+self.batch_size]

            # マスクを3チャンネルに拡張
            batch_masks = batch_masks.unsqueeze(1).repeat(1, 3, 1, 1)

            # マスクを適用した画像の生成
            masked_images = image.repeat(len(batch_masks), 1, 1, 1) * batch_masks

            # 推論実行
            with torch.no_grad():
                predictions = self.model(masked_images)

            # Top1の信頼度を取得してサリエンシーマップを更新
            for j, preds in enumerate(predictions[3]):  # predictions[3]は信頼度のリスト
                if preds:  # 検出結果がある場合
                    top1_conf = preds[0]  # 最も信頼度の高い検出のスコア
                    saliency_map += batch_masks[j, 0] * top1_conf

        # サリエンシーマップの正規化
        saliency_map /= N
        saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min() + 1e-8)

        return saliency_map.cpu().numpy()

def apply_heatmap(image, saliency_map, alpha=0.4):
    """サリエンシーマップのヒートマップ表示"""
    # サリエンシーマップの正規化
    saliency_norm = (saliency_map - np.min(saliency_map)) / (np.max(saliency_map) - np.min(saliency_map))

    # ヒートマップの生成
    heatmap = (saliency_norm * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # サイズの調整
    if image.shape[:2] != heatmap.shape[:2]:
        heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

    # オーバーレイ
    overlayed = cv2.addWeighted(image, 1-alpha, heatmap, alpha, 0)
    return overlayed

def run_rise_analysis(model_path, img_path, num_masks=6000, mask_size=8, mask_prob=0.1, save_path=None):
    """RISE分析の実行"""
    try:
        start_time = time.time()
        device = setup_device()

        # YOLOv5モデルのロード
        print("Loading YOLOv5 model...")
        yolo_model = YOLOV5TorchObjectDetector(
            model_weight=model_path,
            device=device,
            img_size=(640, 640)
        )

        # 画像の読み込みと前処理
        print("Loading and preprocessing image...")
        original_image = Image.open(img_path)
        image_array = np.array(original_image)
        processed_image = yolo_model.preprocessing(np.expand_dims(image_array, 0))

        if processed_image is None:
            raise ValueError("Failed to preprocess image")

        # RISEの初期化（画像サイズを取得後）
        print("Initializing RISE...")
        rise_explainer = YOLORISE(yolo_model)

        # マスクの生成またはロード
        image_h, image_w = processed_image.shape[2:4]
        mask_path = os.path.join(os.path.dirname(save_path) if save_path else '.', f'rise_masks_{image_h}x{image_w}.npy')

        if os.path.exists(mask_path):
            print("Loading pre-generated masks...")
            rise_explainer.load_masks(mask_path)
        else:
            print(f"Generating {num_masks} masks...")
            rise_explainer.generate_masks(savepath=mask_path, image_size=(image_h, image_w))

        # 画像の読み込みと前処理
        print("Loading and preprocessing image...")
        original_image = Image.open(img_path)
        image_array = np.array(original_image)
        processed_image = yolo_model.preprocessing(np.expand_dims(image_array, 0))

        if processed_image is None:
            raise ValueError("Failed to preprocess image")

        # RISEの実行
        print("Running RISE analysis...")
        saliency_map = rise_explainer(processed_image)

        # ヒートマップの生成と保存
        if save_path:
            print("Generating visualization...")
            heatmap = apply_heatmap(image_array, saliency_map)
            cv2.imwrite(save_path, cv2.cvtColor(heatmap, cv2.COLOR_RGB2BGR))

        print(f"Total processing time: {time.time() - start_time:.2f}s")
        return saliency_map

    except Exception as e:
        print(f"Error in RISE analysis: {e}")
        traceback.print_exc()
        return None


# 使用例
if __name__ == "__main__":
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/1.jpg"
    save_path = "rise_analysis.png"

    saliency_map = run_rise_analysis(
        model_path=model_path,
        img_path=img_path,
        save_path=save_path,
        num_masks=6000,
        mask_size=8,
        mask_prob=0.1
    )

/content/yolov5-gradcam/models/experimental.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_download(w), map_location=device)
INFO:models.yolo

Using GPU: Tesla T4
Available GPU memory: 14.75 GB
Loading YOLOv5 model...
[INFO] Loading model...
[INFO] Model loaded successfully
[INFO] Number of classes: 9
[INFO] Using class names: ['infection', 'normal', 'non-infection', 'scar', 'tumor', 'deposit', 'APAC', 'lens opacity', 'bullous']
[INFO] Performing cold start prevention...
[INFO] Initialization complete
Loading and preprocessing image...
Initializing RISE...
Loading pre-generated masks...
Loaded 6000 masks with shape (6000, 448, 640)
Loading and preprocessing image...
Running RISE analysis...
Error in RISE analysis: CUDA out of memory. Tried to allocate 6.41 GiB. GPU 0 has a total capacity of 14.75 GiB of which 4.39 GiB is free. Process 52228 has 10.36 GiB memory in use. Of the allocated memory 7.00 GiB is allocated by PyTorch, and 3.23 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation fo

Traceback (most recent call last):
  File "<ipython-input-19-c20c868d3295>", line 312, in run_rise_analysis
    saliency_map = rise_explainer(processed_image)
  File "<ipython-input-19-c20c868d3295>", line 215, in __call__
    masks_tensor = torch.from_numpy(self.masks).float().to(self.device)
torch.OutOfMemoryError: CUDA out of memory. Tried to allocate 6.41 GiB. GPU 0 has a total capacity of 14.75 GiB of which 4.39 GiB is free. Process 52228 has 10.36 GiB memory in use. Of the allocated memory 7.00 GiB is allocated by PyTorch, and 3.23 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
